In [66]:
import numpy as np
import json
import glob

In [67]:
def load_results(dataset, model, pretrained, ckpt, metric, n_sample, seed):
    if pretrained:
        result_dir = f'../results/{dataset}/pretrained/{model}_{seed}-{ckpt}/{metric}_{n_sample}'
    else:
        result_dir = f'../results/{dataset}/scratch/{model}_{seed}-{ckpt}/{metric}_{n_sample}'
      
    result_fs = glob.glob(f'{result_dir}/*.json')
    results = []
    for file in result_fs:
        with open(file, 'r') as f:
            data = json.load(f)
            print(file, len(data))
        results.extend(data)
        
    return results

In [68]:
n_sample_dict = {
    'CIFAR-10': -1, 
    'CIFAR-100': -1, 
    'Living-17': -1,
    'Nonliving-26': -1,
    'Entity-13': -1,
    'Entity-30': -1,
    'ImageNet': -1,
    'RxRx1': -1,
    'FMoW': -1,
    'Amazon': -1,
    'CivilComments': -1
}

n_epoch_dict = {
    'CIFAR-10': 300, 
    'CIFAR-100': 300, 
    'Living-17': 450,
    'Nonliving-26': 450,
    'Entity-13': 300,
    'Entity-30': 300,
    'ImageNet': 10,
    'FMoW': 50,
    'RxRx1': 90,
    'Amazon': 3,
    'CivilComments': 5
}

pretrained_dict = {
    'CIFAR-10': False, 
    'CIFAR-100': False, 
    'Living-17': False,
    'Nonliving-26': False,
    'Entity-13': False,
    'Entity-30': False,
    'ImageNet': True,
    'FMoW': True,
    'RxRx1': True,
    'Amazon': True,
    'CivilComments': True
}

In [137]:
metric = 'COTT-L1'
dataset = 'CIFAR-10'
arch = 'resnet18'
n_sample = n_sample_dict[dataset]
seed = '1'
model_ckpt = n_epoch_dict[dataset]
pretrained = pretrained_dict[dataset]
results = load_results(dataset, arch, pretrained, model_ckpt, metric, n_sample, seed)

../results/CIFAR-10/scratch/resnet18_1-300/COTT-L1_-1/clean.json 1
../results/CIFAR-10/scratch/resnet18_1-300/COTT-L1_-1/collection.json 1
../results/CIFAR-10/scratch/resnet18_1-300/COTT-L1_-1/brightness.json 5
../results/CIFAR-10/scratch/resnet18_1-300/COTT-L1_-1/defocus_blur.json 5
../results/CIFAR-10/scratch/resnet18_1-300/COTT-L1_-1/elastic_transform.json 5
../results/CIFAR-10/scratch/resnet18_1-300/COTT-L1_-1/fog.json 5
../results/CIFAR-10/scratch/resnet18_1-300/COTT-L1_-1/frost.json 5
../results/CIFAR-10/scratch/resnet18_1-300/COTT-L1_-1/gaussian_blur.json 5
../results/CIFAR-10/scratch/resnet18_1-300/COTT-L1_-1/gaussian_noise.json 5
../results/CIFAR-10/scratch/resnet18_1-300/COTT-L1_-1/glass_blur.json 5
../results/CIFAR-10/scratch/resnet18_1-300/COTT-L1_-1/impulse_noise.json 5
../results/CIFAR-10/scratch/resnet18_1-300/COTT-L1_-1/jpeg_compression.json 5
../results/CIFAR-10/scratch/resnet18_1-300/COTT-L1_-1/motion_blur.json 5
../results/CIFAR-10/scratch/resnet18_1-300/COTT-L1_-1/p

In [138]:
len(results)

97

In [139]:
import altair as alt
import pandas as pd

In [140]:
def get_corr_chart(data, subpop):
    corr = alt.Chart(alt.Data(values=data), title=subpop).mark_point(size=50).encode(
        x=alt.X('metric:Q', title=metric),
        y=alt.X('error:Q', title='Test Error'),
        color=alt.Color('corruption:N', scale=alt.Scale(scheme='category20')),
        shape='corruption level:O'
    ).properties(
        width=200,
        height=200
    )
    return corr

In [141]:
same_pop_results =  [i for i in results if i['subpopulation'] == 'same']
natural_pop_results =  [i for i in results if i['subpopulation'] == 'natural']
novel_pop_results = [i for i in results if i['subpopulation'] == 'novel']

In [142]:
line = pd.DataFrame({'metric': [0, 1], 'error': [0, 1]})
line_plot = alt.Chart(line).mark_line(color='black', strokeDash=[5, 8]).encode(
    x='metric',
    y='error',
)

In [143]:
same_corr = get_corr_chart(same_pop_results, 'same')
same_plt = same_corr + line_plot

natural_corr = get_corr_chart(natural_pop_results, 'natural')
natural_plt = natural_corr + line_plot

novel_corr = get_corr_chart(novel_pop_results, 'novel')
novel_plt = novel_corr + line_plot

plt = same_plt | natural_plt | novel_plt

In [144]:
plt.configure_axis(
    labelFontSize=14,
    titleFontSize=16
).configure_axis(
    labelFontSize=14,
    titleFontSize=16,
).configure_legend(
    titleFontSize=14,
    labelFontSize=16
) 


/usr/workspace/lu35/anaconda3/envs/ood/lib/python3.9/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.HConcatChart(...)

In [145]:
def polyfit(x, y, degree=1):
    results = {}

    coeffs = np.polyfit(x, y, degree)

    results['polynomial'] = coeffs.tolist()

    p = np.poly1d(coeffs)

    yhat = p(x)                
    ybar = np.sum(y)/len(y)          
    ssreg = np.sum((yhat - ybar)**2)   
    sstot = np.sum((y - ybar)**2)    
    results['determination'] = ssreg / sstot

    return results

In [146]:
from scipy.stats import spearmanr
import math

In [147]:
def compute_corr_stats(results):
    if len(results) == 0:
        return
    
    d = [i['metric'] for i in results]
    e = [i['error'] for i in results]
    
    if len(results) > 1:
        print(f'{metric} r2:', polyfit(d, e)['determination'])

        coef, p = spearmanr(d, e)
        print(f'{metric} rho:', coef)

        print(f'{metric} slope:', polyfit(d, e)['polynomial'][0])
        print(f'{metric} bias:', polyfit(d, e)['polynomial'][1])

    yhat = np.array(d)
    y = np.array(e)

    print('MAE:', np.abs(yhat - y).mean() * 100)

In [148]:
print("----- same pop results -----")
compute_corr_stats(same_pop_results)

----- same pop results -----
COTT-L1 r2: 0.9913768375091828
COTT-L1 rho: 0.9961916324490862
COTT-L1 slope: 0.9291552779029837
COTT-L1 bias: 0.0005174339920514121
MAE: 2.4534356981515875


In [149]:
print("----- natural pop results -----")
compute_corr_stats(natural_pop_results)

----- natural pop results -----
MAE: 2.920004100799561


In [150]:
print("----- novel pop results -----")
compute_corr_stats(novel_pop_results)

----- novel pop results -----


In [151]:
print("----- novel pop results -----")
compute_corr_stats([i for i in novel_pop_results if i['corruption level'] == 0 ])

----- novel pop results -----
